#Inspiration

https://dev.to/devdevcharlie/play-street-fighter-with-body-movements-using-arduino-and-tensorflow-js-4kbi

Built in JS with Johnny-Five and TensorFlow.js, goal is to port it to TensorFlow Lite Micro and run ML inferencing directly on an Arduino.

# Arduino setup

1. Download the Arduino IDE
1. Install Nano 33 BLE board support via: `Tools -> Board: ... -> Board Manager ...`
1. Install the `Arduino_LSM9DS1` library via: `Sketch -> Include Library -> Manage Libraries ...`
1. Upload the sketch below and open the Serial Monitor.
1. Press the button to start recording the gestue for 1 second.
1. Repeat the above step many times, copy the output of the Serial Monitor and save as a ``.csv` file.
1. Repeat the for all the gestures you want to train.
1. Copy data into the notebook code cells prefixed with `%%writefile <gesture name>.csv`



# Arduino Sketch - Record Data

> Indented block


```arduino
#include <Arduino_LSM9DS1.h>

const int buttonPin = 3;     // the number of the pushbutton pin
const int ledPin =  13;      // the number of the LED pin
const int numSamples = 119;

int previousButtonState = HIGH;
int samplesRead = numSamples;

void setup() {
  Serial.begin(9600);
  while (!Serial);

  // initialize the LED pin as an output:
  pinMode(ledPin, OUTPUT);
  // initialize the pushbutton pin as an input with pullup:
  pinMode(buttonPin, INPUT_PULLUP);

  if (!IMU.begin()) {
    Serial.println("Failed to initialize IMU!");
    while (1);
  }

  Serial.println("aX,aY,aZ,gX,gY,gZ");
}

void loop() {
  int buttonState = digitalRead(buttonPin);

  if (buttonState != previousButtonState) {
    if (buttonState == LOW) {
      if (samplesRead == numSamples) {
        // pressed
        samplesRead = 0;
      }
    } else {
      // released
    }

    previousButtonState = buttonState;
  }

  if (samplesRead < numSamples) {
    if (IMU.accelerationAvailable() && IMU.gyroscopeAvailable()) {
      float aX, aY, aZ, gX, gY, gZ;

      IMU.readAcceleration(aX, aY, aZ);
      IMU.readGyroscope(gX, gY, gZ);

      samplesRead++;

      Serial.print(aX, 3);
      Serial.print(',');
      Serial.print(aY, 3);
      Serial.print(',');
      Serial.print(aZ, 3);
      Serial.print(',');
      Serial.print(gX, 3);
      Serial.print(',');
      Serial.print(gY, 3);
      Serial.print(',');
      Serial.print(gZ, 3);
      Serial.println();

      if (samplesRead == numSamples) {
        Serial.println();
      }
    }
  }
}
```



# Setup Python Environment 

The next cell sets up the dependencies in required for the notebook, run it.

In [0]:
# Setup environment
!apt-get -qq install xxd
!pip install tensorflow==2.0.0-rc1

# Gather Data

1. Run the Arduino Code
1. Push the button, make a punch gesture
1. Repeat 10x
1. Copy and paste the data from the serial output to punch.csv
1. Clear the console data
1. Push the button, flex
1. Repeat 10x
1. Copy and paster the serial output to flex.csv



# Upload Data

1. Open the panel on the left side of Colab by clicking on the __>__
1. Select the files tab
1. Drag `punch.csv` and `flex.csv` files from your computer to the tab to upload them into colab.


# TODO
Add a test for uploaded models here

List the file names in /content/*.csv

Throw an error if we don't see any

# Train Neural Network

The next cell parses the csv and trains a fully connected neural network.

Update the `GESTURES` list with the gesture data you've collected in `.csv` format.

The models performance vs validation will also be graphed.


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print("TensorFlow version =", tf.__version__)

# set a fixed random seed for reproducibility
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# the list of gestures that data is available for
GESTURES = [
    "punch",
    "flex",
]

SAMPLES_PER_GESTURE = 119

NUM_GESTURES = len(GESTURES)

# create a one-hot encoded matrix that is used in the output
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# read each csv file and push an input and output
for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(gesture_index, gesture)
  
  output = ONE_HOT_ENCODED_GESTURES[gesture_index]
  
  df = pd.read_csv("/content/" + gesture + ".csv")
  
  # TODO this name is confusing because of NUM_GESTURES
  # maybe num_samples?
  num_gestures = int(df.shape[0] / SAMPLES_PER_GESTURE)
  
  print(num_gestures)
  
  for i in range(num_gestures):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      tensor += [
          df['aX'][index],
          df['aY'][index],
          df['aZ'][index],
          df['gX'][index],
          df['gY'][index],
          df['gZ'][index]
      ]

    inputs.append(tensor)
    outputs.append(output)

    
inputs = np.array(inputs)
outputs = np.array(outputs)


# Randomize the order of the inputs
# frome: https://stackoverflow.com/a/37710486/2020087
# TODO this might be a better way to do the randomization https://stackoverflow.com/a/30633632
num_inputs = len(inputs)
randomize = np.arange(num_inputs)

np.random.shuffle(randomize)

inputs = inputs[randomize]
outputs = outputs[randomize]

# TODO are we splitting each group of 119 samples or just rows of data?
# spit the data into three bins: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)


inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

# build the model and train it
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(15, activation='softmax'))
model.add(tf.keras.layers.Dense(NUM_GESTURES))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=600, batch_size=1, validation_data=(inputs_validate, outputs_validate))

# graph the loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# graph the loss again skipping a bit of the start
SKIP = 100
plt.plot(epochs[SKIP:], loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# graph of mean absolute error
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs[SKIP:], mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()


# use the model to predict the test inputs
predictions = model.predict(inputs_test)

# print the predictions and the expected ouputs
print(predictions)
print(outputs_test)


# Plot the predictions along with to the test data
plt.clf()
plt.title('Training data predicted vs actual values')
plt.plot(inputs_test, outputs_test, 'b.', label='Actual')
plt.plot(inputs_test, predictions, 'r.', label='Predicted')
# plt.legend()
plt.show()

# Convert Trained Model to Tensor Flow Light

The next cell converts the model to TFlite format. It also creates a quantized model, that we'll ignore for now. The size in bytes of each model is also printed out.

In [0]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)
# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
# Save the model to disk
open("gesture_model_quantized.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("gesture_model_quantized.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)
  
  

Basic model is 147764 bytes
Quantized model is 40712 bytes
Difference is 107052 bytes


## Encode the Model in an Arduino Header File 

The next cell creates a constant byte array that contains the TFlite model. Import it as a tab with the sketch below.

In [0]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel. Double click model.h to download the file.")

Header file, model.h, is 911,246 bytes.

Open the side panel. Double click model.h to download the file.


# Arduino Sketch - Detect Gestures
```
#include <Arduino_LSM9DS1.h>

#include <TensorFlowLite.h>
#include "tensorflow/lite/experimental/micro/kernels/all_ops_resolver.h"
#include "tensorflow/lite/experimental/micro/micro_error_reporter.h"
#include "tensorflow/lite/experimental/micro/micro_interpreter.h"
#include "tensorflow/lite/schema/schema_generated.h"
#include "tensorflow/lite/version.h"

#include "model.h"

const int buttonPin = 3;     // the number of the pushbutton pin
const int ledPin =  13;      // the number of the LED pin
const int numSamples = 119;

int previousButtonState = HIGH;
int samplesRead = numSamples;

// Globals, used for compatibility with Arduino-style sketches.
tflite::ErrorReporter* g_error_reporter = nullptr;
const tflite::Model* g_model = nullptr;
tflite::MicroInterpreter* g_interpreter = nullptr;
TfLiteTensor* g_input = nullptr;
TfLiteTensor* g_output = nullptr;
int g_inference_count = 0;

// Create an area of memory to use for input, output, and intermediate arrays.
// Finding the minimum value for your model may require some trial and error.
constexpr int g_tensor_arena_size = 8 * 1024;
uint8_t g_tensor_arena[g_tensor_arena_size];

const char* GESTURES[] = {
  "punch",
  "flex"
};

#define NUM_GESTURES (sizeof(GESTURES) / sizeof(GESTURES[0]))

// The name of this function is important for Arduino compatibility.
void setup() {
  Serial.begin(9600);
  while (!Serial);

  // initialize the LED pin as an output:
  pinMode(ledPin, OUTPUT);
  // initialize the pushbutton pin as an input with pullup:
  pinMode(buttonPin, INPUT_PULLUP);

  if (!IMU.begin()) {
    Serial.println("Failed to initialize IMU!");
    while (1);
  }

  Serial.print("Accelerometer sample rate = ");
  Serial.print(IMU.accelerationSampleRate());
  Serial.println(" Hz");
  Serial.print("Gyroscope sample rate = ");
  Serial.print(IMU.gyroscopeSampleRate());
  Serial.println(" Hz");

  Serial.println();

  // Set up logging
  static tflite::MicroErrorReporter micro_error_reporter;
  g_error_reporter = &micro_error_reporter;

  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  g_model = tflite::GetModel(model);
  if (g_model->version() != TFLITE_SCHEMA_VERSION) {
    g_error_reporter->Report(
      "Model provided is schema version %d not equal "
      "to supported version %d.\n",
      g_model->version(), TFLITE_SCHEMA_VERSION);
    return;
  }

  // This pulls in all the operation implementations we need
  static tflite::ops::micro::AllOpsResolver resolver;

  // Build an interpreter to run the model with
  static tflite::MicroInterpreter interpreter(
    g_model, resolver, g_tensor_arena, g_tensor_arena_size, g_error_reporter);
  g_interpreter = &interpreter;

  // Allocate memory from the tensor_arena for the model's tensors
  g_interpreter->AllocateTensors();

  // Obtain pointers to the model's input and output tensors
  g_input = g_interpreter->input(0);
  g_output = g_interpreter->output(0);


  // Keep track of how many inferences we have performed
  g_inference_count = 0;
}

// The name of this function is important for Arduino compatibility.
void loop() {
  int buttonState = digitalRead(buttonPin);

  if (buttonState != previousButtonState) {
    if (buttonState == LOW) {
      if (samplesRead == numSamples) {
        // pressed       }
        samplesRead = 0;
      }
    } else {
      // released
    }

    previousButtonState = buttonState;
  }

  if (samplesRead < numSamples) {
    if (IMU.accelerationAvailable() && IMU.gyroscopeAvailable()) {
      float aX, aY, aZ, gX, gY, gZ;

      IMU.readAcceleration(aX, aY, aZ);
      IMU.readGyroscope(gX, gY, gZ);

      g_input->data.f[samplesRead * 6 + 0] = aX;
      g_input->data.f[samplesRead * 6 + 1] = aY;
      g_input->data.f[samplesRead * 6 + 2] = aZ;
      g_input->data.f[samplesRead * 6 + 3] = gX;
      g_input->data.f[samplesRead * 6 + 4] = gY;
      g_input->data.f[samplesRead * 6 + 5] = gZ;

      samplesRead++;

      if (samplesRead == numSamples) {
        // Run inference, and report any error
        TfLiteStatus invoke_status = g_interpreter->Invoke();
        if (invoke_status != kTfLiteOk) {
          g_error_reporter->Report("Invoke failed on x_val: %f\n",
                                   static_cast<double>(0.0));
          while (1);
          return;
        }

        // Read the predicted y value from the model's output tensor
        float y_val = g_output->data.f[0];

        for (int i = 0; i < NUM_GESTURES; i++) {
          Serial.print(GESTURES[i]);
          Serial.print(": ");
          Serial.println(g_output->data.f[i], 6);
        }
        Serial.println();
      }
    }
  }
}

```

## Arduino BLE Keyboard

TODO Arduino / BLE HID code goes here


# Add More Gestures

Now that you have this working... Load the code to record gestures. Create more CSV files with gestures. Retrain the model. Load the new model back onto the Arduino.

Note: you'll need to edit the code to add the names of the new geture files.